#### Combine both models (FP-Growth + ALS)

##### Author - Reshma

ECFAR adopts the rules-based recommendation strategy for market basket recommendations. Therefore, AR and CFP generated by the previous steps are combined for the final recommendation.

In [3]:
import pandas as pd
import numpy as np
import multiprocessing as mp
import time as time

In [ ]:
### functions

# association_rule
def convert_to_arr(arr_string):
#     print(arr_string[0])
    str1 = arr_string.replace(']','').replace('[','')
    l    = str1.split(",")
    return [int(e) for e in l]

def get_consequent(list_item):
    conse_list = []
    for item in list_item:
        association_rule['select'] = association_rule['antecedent'].apply(lambda x : 1 if item in x else 0)
        df_part = association_rule[association_rule['select'] ==1]
        if df_part.shape[0]>0:
            df_part.sort_values(by=['confidence'], ascending=False, inplace=True)
            cons_item = df_part['consequent'].values[0]
            conse_list.append(cons_item[0])
    return list(set(conse_list))

def get_collab_pred(user_id, len_):
    if len_ <15:
        return collab_gr[collab_gr['user_id']==user_id]['db_food_id'].values[0][0:len_]
    else:
        return []
    
def get_actual_pred(user_id):
    return test_basket[test_basket['user_id']==user_id]['db_food_id'].values[0]

#### Calculate hit-ratio
def calc_hit_ratio(preds, actual):
    len_act = len(actual)
    hits_   = list(set(preds).intersection(set(actual)))
    return (len(hits_)/len_act)

def confi_cut(score):
    if score >= 0.0 and score <0.2:
        return '0-0.2'
    elif score >= 0.2 and score <0.4:
        return '0.2-0.4'
    elif score >= 0.4 and score <0.6:
        return '0.4-0.6'
    elif score >= 0.6 and score <0.8:
        return '0.6-0.8'
    else:
        return '0.8-1.0'

In [2]:
association_rule= pd.read_csv('association_rule_results.csv')
collab_filter   = pd.read_csv('collab_filter_results.csv')

In [8]:
collab_filter['db_food_id'] = collab_filter['db_food_id'].map(int)

In [9]:
collab_gr = collab_filter.groupby('user_id')['db_food_id'].apply(list).reset_index()

In [14]:
association_rule['antecedent'] = association_rule['antecedent'].apply(lambda x: convert_to_arr(x))
association_rule['consequent'] = association_rule['consequent'].apply(lambda x: convert_to_arr(x))

In [15]:
association_rule.head(10)

,antecedent,consequent,confidence,lift
0,[32884],[43562],0.000135,1.087669
1,[32884],[18918],0.000135,1.814305
2,[32884],[28159],0.000135,0.659244
3,[32884],[19778],0.000135,1.370808
4,[32884],[56070],0.000135,1.121570
5,[32884],[37266],0.000135,1.124491
6,[32884],[55069],0.000135,2.096139
7,[32884],[54835],0.000135,1.160765
8,[32884],[17259],0.000135,1.936344
9,[32884],[14848],0.000135,1.183026


In [24]:
# Import Data
dataDir = "/scratch/midway2/reshmask/"
data    = pd.read_csv(dataDir + "instacart_dataset_spark.csv")

In [25]:
## split train and test dataset
train_df = data[data["eval_set"]=="prior"]
test_df  = data[data["eval_set"]=="train"]

In [26]:
## groupby test basket
train_basket = train_df.groupby('user_id')['db_food_id'].apply(list).reset_index()
test_basket  = test_df.groupby('user_id')['db_food_id'].apply(list).reset_index()

In [8]:
## Get train set K frequent items

In [8]:
k= 15

In [9]:
from collections import Counter
def get_topK_items(list_items):
    most_cmn = Counter(list_items).most_common(k)
    return ([ i[0] for i in most_cmn])

In [10]:
train_basket['top_k_items']     = train_basket['db_food_id'].apply(lambda x : get_topK_items(x))

In [11]:
train_basket['len_top_k_items'] = train_basket['top_k_items'].apply(lambda x : len(x))

In [14]:
p = mp.Pool(mp.cpu_count()) # parallel processing

/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/TF_latest_GPU/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
train_basket['assoc_Prediction'] = p.map(get_consequent, train_basket['top_k_items'])

In [ ]:
train_basket['len_Prediction'] = = train_basket['assoc_Prediction'].apply(lambda x : len(x))

In [ ]:
## Combine collaborative filtering result if len of prediction is less than 15
train_basket['collab_Prediction'] = train_basket[['user_id','len_prediction']].apply(lambda x : get_collab_pred(x))

In [ ]:
train_basket['final_Prediction']  = train_basket['assoc_Prediction'] + train_basket['collab_Prediction']

In [28]:
train_basket['actual_Prediction'] =  train_basket['user_id'].apply(lambda x : get_actual_pred(x))

In [ ]:
train_basket['Hit-Ratio'] = train_basket[['final_prediction', 'actual_prediction']].apply(lambda x: calc_hit_ration(x[0], x[1]))

In [4]:
train_basket['confi'] = train_basket['Hit-Ratio'].apply(lambda x: confi_cut(x))
train_basket['confi'].value_counts(normalize=True)

,confi
0-0.2,0.3012
0.2-0.4,0.2697
0.4-0.6,0.2603
0.6-0.8,0.0911
0.8-1.0,0.0833
